# 🚲 ¿Qué hace que alquilemos una bicicleta?  
### Un análisis y modelo predictivo de alquiler de bicicletas

Este proyecto analiza los factores que influyen en el número de bicicletas alquiladas en una ciudad a lo largo del día.  
Exploraremos el comportamiento de los usuarios, realizaremos limpieza y visualizaciones de los datos, y construiremos un modelo predictivo usando regresión lineal.



1. Introducción
2. Objetivo del proyecto
2. Exploración del Dataset
3. Limpieza de datos
4. Análisis Exploratorio de Datos (EDA)
5. Correlaciones 
7. Modelo
6. Conclusiones

## 1. Introducción


![Predicción de Alquiler de bicicletas](Etbycle.JPG)

En una de mis visitas a Bogotá noté que cada vez más personas usan la bicicleta como medio de transporte. Me pareció una alternativa amigable con el medio ambiente, pero también me surgieron varias preguntas: ¿Las bicicletas son propias o alquiladas? ¿Qué factores influyen en la decisión de salir en bicicleta al trabajo u otros lugares?

Para responder estas preguntas, trabajé con un dataset de Kaggle sobre alquiler de bicicletas. Aunque los datos corresponden a una ciudad específica, permiten analizar patrones que pueden extrapolarse a otras ciudades donde el uso de la bicicleta está en crecimiento.

Este análisis incluye:
- Un análisis exploratorio de los datos
- Visualización de relaciones clave
- Modelado de regresión lineal simple y múltiple para predecir la demanda


## 2. Objetivo del Proyecto

El objetivo principal de este proyecto es construir modelos de regresión lineal simple y múltiple que permitan **predecir la cantidad de bicicletas alquiladas**, teniendo en cuenta variables como la hora del día, el clima, la temperatura y otros factores contextuales.

Este tipo de análisis permite entender mejor los patrones de comportamiento de los usuarios y puede ayudar a optimizar la planificación y distribución de recursos en los servicios de alquiler de bicicletas.


El dataset utilizado es Bike Sharing Demand, extraído de Kaggle.

Período: 2 años
Observaciones: 17.379 registros
Variables: 16 columnas

Las filas representan un dato horario del número de bicletas alquiladas, con otras variables temporales y metorológicas.

Respecto a las variables que se encuentran en el archivo les muestro lo que significa cada columna de estos datos:

* **datetime:** La fecha y hora del registro, que permite analizar patrones por día, semana o estación.

* **season:** Estación del año (primavera, verano, etc.)

* **holiday:** Día festivo.

* **workingday:** Si fue un día laboral o no.

* **weather:** Un índice de la condición climática, desde claro hasta lluvia intensa.

* **temp / atemp:** Temperatura real y sensación térmica.

* **humidity:** Humedad relativa.

* **windspeed:** Velocidad del viento.

* **casual / registered:** Alquileres por usuarios casuales (espontáneos) y registrados (frecuentes).

* **count:** El total de bicicletas alquiladas en ese momento (Variable objtivo).

Este proyecto toma como base un ejercicio del Bootcamp de Data Science de Digital House,a partir de ese punto de partida, desarrollé un análisis exploratorio, visualizaciones personalizadas y un modelo de regresión para predecir la demanda de bicicletas según las condiciones climáticas y temporales.


### 3. Exploración del Dataset

In [ ]:
# Importamos la librería pandas para visualizar en modo de tabla en este notebook nuestros datos

import pandas as pd

# ruta: Creamos una variable para almacenar la ubicación del archivo CSV, que nos ayuda acortar el codigo.

data_bikes = r"D:\Git Hub\Regresión simple y múltiple\Alquiler de Bicicletas\bikes.csv"

# Leemos el archivo CSV utilizando la función read_csv de pandas y lo cargamos en un DataFrame llamado: data.
data = pd.read_csv(data_bikes, index_col='datetime', parse_dates=True)

#Se convirtió la columna datetime al formato de fecha y hora

# Utilizamos el método head(5) para mostrar las primeras 5 filas del DataFrame 'data', lo que nos da
# una vista rápida del contenido del dataset.
data.head(5)



In [ ]:
#Se revisaron los valores nulos y se confirmó que no había datos faltantes
data.info()

In [ ]:
# Estadísticas descriptivas

#count (número de datos)

#mean (media)

#std (desviación estándar)

#min, 25%, 50%, 75%, max


data.describe()


In [ ]:
#Número de filas y columnas del dataset
data.shape

In [ ]:
data.sample(10)

### 4. Limpieza de Datos

In [ ]:
#Count en pandas es un método por lo que por buenas prácticas es recomendable cambiar el nombre de la columna
data.rename(columns={'count':'totally'}, inplace=True)
data.sample(10)

In [ ]:
#Validación de nulls

data.info()
data.isnull().sum()


In [ ]:
#Revisión de duplicados

print(f"Duplicados: {data.duplicated().sum()}")


### 5. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Visaulización de la variable objetivo

# Importamos las librerías
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Visualización de la variable objetivo (total de bicicletas alquiladas)
sns.histplot(data['totally'], kde=True, color='orchid')
plt.title('Distribución de alquileres totales')
plt.xlabel('Bicicletas alquiladas')
plt.ylabel('Frecuencia')
plt.show()


La distribución de los alquileres de las bicicletas presenta una ligera asimetría a la derecha, por lo que deducimos la mayoría de veces el número de bicicletas alquiladas es bajo a medio, pero hay días con alta demanda.

In [ ]:
# Alquiler estación

sns.boxplot(x='season', y='totally', data=data)
plt.title('Alquiler de bicicletas por estación')
plt.xlabel('Estación')
plt.ylabel('Total de alquileres')
plt.show()


Las estaciones con mayor alquiler son la 2 y 3 que concuerda con primavera y verano lo que asu vez se asume favorable por las condiciones del clima y el uso de bicicletas. 

In [ ]:
#Extraemos la hora

data['hour'] = data.index.hour

# Promedio de alquileres por hora del día
sns.lineplot(x='hour', y='totally', data=data.groupby('hour')['totally'].mean().reset_index())
plt.title('Promedio de alquileres por hora del día')
plt.xlabel('Hora')
plt.ylabel('Alquiler promedio')
plt.show()


Observamos 2 picos que indican mayor demanda de bicicletas aproximadamente a las 8 am y otro a las 5:30 pm que pod´ria alinearse con horarios laborales o escolares. 

In [ ]:
#Variables cagóricas
#  Comparación de alquileres en días festivos vs normales
sns.boxplot(x='holiday', y='totally', data=data)
plt.title('Alquileres en días festivos vs normales')


Los días festivos están un poco por debajo de los días normales, puede deberse a que menos personas deben ir a trabajar estos días.

In [ ]:
# Identificación de Outliers en humedad
sns.boxplot(x=data['humidity'])
plt.title('Outliers en Humedad')


Algunos valores extremos de humedad pueden afectar el alquiler de bicicletas



In [ ]:
###Vamos a visualizar las variables clave de nuestro Datset


# Parámetros visuales globales
plt.rcParams['figure.figsize'] = (8, 6) #tamaño e los gráficos
plt.rcParams['font.size'] = 14 #tamaño de la fuente
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Verdana'] #tipo de letra

# Estilo de seaborn
sns.set_style("whitegrid") #fondo blanco
sns.set_palette("pastel") #colores

# 1. Temperatura vs Alquiler (Gráfica de dispersión simple)
data.plot(kind='scatter', #gráfico de dispersión
    x='temp', #eje x
    y='totally', #eje y el total de bicletas alquilas
    alpha=0.2, #transparencia de puntos
    color='mediumslateblue' # color de los puntos
)
plt.title('Relación de temperatura y alquiler de Bicicletas', fontsize=16)
plt.xlabel('Temperatura (°C)', fontsize=12)
plt.ylabel('Total de bicicletas alquiladas', fontsize=12)
plt.show()


#Gráfica con línea de tendecia 

sns.lmplot( #Modelo de regresión
    x='temp', 
    y='totally', 
    data=data, 
    aspect=1.45, #ancho y alto
    scatter_kws={'alpha': 0.3, 'color': 'orchid'}, #visualización de los puntos
    line_kws={'color': 'darkviolet', 'linewidth': 2}
)
plt.title('Regresión de Temperatura vs Alquiler de bicicletas', fontsize=16)
plt.xlabel('Temperatura (°C)', fontsize=12)
plt.ylabel('Alquiler de Bicicletas', fontsize=12)
plt.show()


A mayor temperatura, mayor número de alquileres, aunque no completamente lineal.

In [ ]:
# 2. Humedad vs Alquiler
sns.lmplot(
    x='humidity', 
    y='totally', 
    data=data, 
    aspect=1.45,
    scatter_kws={'alpha': 0.3, 'color': 'skyblue'},
    line_kws={'color': 'deepskyblue', 'linewidth': 2}
)
plt.title('Humedad en el aire, ¿menos bicis en la calle?', fontsize=16)
plt.xlabel('Humedad (%)', fontsize=12)
plt.ylabel('Total de bicicletas alquiladas', fontsize=12)
plt.show()


Podemos observar una relación inversamente proporcional, a mayor humedad menor uso de bicicletas y tiene sentido ya que éste estado del clima hace que sea incomodo usar bicileta.

In [ ]:
# 3. Velocidad del viento vs Alquiler
sns.lmplot(
    x='windspeed', 
    y='totally', 
    data=data, 
    aspect=1.45,
    scatter_kws={'alpha': 0.3, 'color': 'lightcoral'},
    line_kws={'color': 'firebrick', 'linewidth': 2}
)
plt.title('Relación entre el viento y el alquiler', fontsize=16)
plt.xlabel('Velocidad del viento (km/h)', fontsize=12)
plt.ylabel('Total de bicicletas alquiladas', fontsize=12)
plt.show()

Se puede observar que a mayor velocidad del viento se empieza a reducir el uso de bicicletas

### 6. Correlaciones

In [ ]:
# Correlaciones entre variables

plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

Se observan correlaciones positivas con la temperatura (temp) y negativas con la humedad (humidity), de lo que se deduce que el buen clima hace que la gente rente mas bicicletas y la humedad que se disminuya. 
También es posible observa la multicolinealidad de temp y atemp por que en este modelo líneal podemos quedarnos con sólo una.


### 7.Evaluación del modelo

El objetivo de éste notebook es analizar la relación entre la temperatura y el número total de bicicletas alquiladas (totally) usando un modelo de regresión en este caso el **Modelo de regresión lineal simple**

El modelo está dado por: 

Y = B0 + B1x1 + B2x2 + B3x3 + ... + Bnxn 

Donde Y = Variable dependiente en este caso totally (lo que queremos hallar)
xn = Variables independiente o features, en este caso temperaturas
B0 = Intercepto, valor de y cuando todos los xi=0
Bn = coeficientes que el modelo va aprender (Aprenden usando el criterio de mínimos cuadrados), indica cuánto cambia Y por un cambuo en xi

Este modelo busca la línea mas óptima, osea la que minimiza el error cuadrático.

Usaremos Scikit-Learn para construir el modelo de regresión lineal, es importante conocer un paso a paso:

    1. Seleccionar la variable objetivo (y-totally) y la feature o varibales predictoras (x-temperatura)
    2. Importar el modelo a usar o sea el de regresión lineal
    3. Instanciar el modelo y seleccionar los hiperparámetros
    4. Ajustar nuestro modelo a los datos con el método **fit()**
    5. Obtener los coeficientes para interpretar la relación

In [ ]:
# 1.Seleccionar Features y target

feature_cols = ['temp']  # Feature independiente
X = data[feature_cols]
y = data['totally']  # Variable dependiente


X.shape

In [ ]:
from sklearn.linear_model import LinearRegression

# 2. Importamos el paquete e instanciamos el modelo
linreg = LinearRegression()

# 3. Ajustamos del modelo
linreg.fit(X, y)

# 4. Vista de los coeficientes
print("Intercepto (β0):", linreg.intercept_)
print("Coeficiente (β1) para temp:", linreg.coef_)


Intercepto (β0): Es el valor esperado de alquileres de bicicletas cuando la temperatura es 0 °C, en donde la recta de la regresión corta el eje y

Coeficiente para temp (β1): Nos dice cuánto cambia la cantidad de  bicicletas alquiladas por cada grado adicional de temperatura.
Ejemplo: si β1 = 9.17, entonces subir 1 °C se asocia con +9.17 bicicletas alquiladas.
Si este coeficiente fuera negativo nos indicaría que a mayor temperatura menor alquileres.

In [ ]:
#Ahora, queremos saber cuántas bicicletas se pueden alquilar a una temperatura específica

# Supongamos una temperatura de 23 grados
test = 23

# Usamos la fórmula del modelo: y = β₀ + β₁ * x
linreg.intercept_ + linreg.coef_ * test


In [ ]:
# Predicción manual
pred_manual = linreg.intercept_ + linreg.coef_[0] * temp_test
print(f"Predicción manual para 23°C: {pred_manual:.0f}")

In [ ]:
# Predicción usando sklearn

import numpy as np

# Creamos un array con la temperatura
test_sklearn = np.array(test).reshape(-1, 1)

# Hacemos la predicción con el modelo
linreg.predict(test_sklearn)


In [ ]:
temp_array = np.array([[temp_test]])
pred_sklearn = linreg.predict(temp_array)[0]
print(f"Predicción sklearn para 23°C: {pred_sklearn:.0f}")

In [ ]:
pred = linreg.predict(test_sklearn)[0]
print(f"A 23°C se espera que se alquilen aproximadamente {pred:.0f} bicicletas.")


In [ ]:
# Si la temperatura es en otras unidades...
# Creamos una nueva columna para la temperatura en Fahrenheit
data['temp_F'] = data.temp * 1.8 + 32
data.head()


In [ ]:
sns.lmplot(x='temp_F', y='totally', data=data, aspect=1.45, scatter_kws={'alpha':0.2});


In [ ]:
from sklearn.linear_model import LinearRegression

feature_cols = ['temp_F']
X = data[feature_cols]
y = data.totally

# Instanciamos el modelo y lo entrenamos
linreg = LinearRegression()
linreg.fit(X, y)

# Mostramos el intercepto y el coeficiente
print("Intercepto:", linreg.intercept_)
print("Coeficiente para temp_F:", linreg.coef_[0])


In [ ]:
test_en_f = 25 * 1.8 + 32
test_sklearn_en_f = np.array(test_en_f).reshape(-1, 1)
pred = linreg.predict(test_sklearn_en_f)[0]

print(f"A 77°F (25°C) se predicen aproximadamente {pred:.0f} alquileres.")


In [ ]:
#Limpiamos la columna 
data.drop('temp_F', axis=1, inplace=True)


In [ ]:
#Que otras variables podemos observar 
# Elegimos nuevas variables para explorar su relación con 'totally'
feature_cols = ['temp', 'season', 'weather', 'humidity']

# Visualizamos regresiones simples entre cada feature y el target
sns.pairplot(data, x_vars=feature_cols, y_vars='totally', kind='reg', height=5, aspect=1);


In [ ]:
# Matriz de correlación
data.corr()


In [ ]:
#Mapa de calor
sns.heatmap(data.corr(), vmin=-1, vmax=1, center=0, cmap="YlGnBu", annot=True);


Observamos la relación de otras varibales como temperatura, humedad, estación y clima con la cantidad total de alquileres.

Utilicé pairplot para ver regresiones simples y tendencias en los datos y una matriz de correlación para medir la fuerza de esas relaciones.

Concluimos que la temperatura tiene una correlación positiva lo cuál es lógico, a mayor temperatura mayor uso de bicicletas.

A mayor humedad menor alquiler de bicicletas (correlación negativa).

El clima y la estación también muestran una relación pero menos fuerte que la temperatura.


In [ ]:
# Regresión lineal múltiple

#Usar multiples features
#Analizamos como influyen otras variables no sólo la temperatura
# Seleccionamos varias features para el modelo
feature_cols = ['temp', 'humidity']

# Creamos X e y
X = data[feature_cols]
y = data.totally

# Instanciamos y ajustamos el modelo
linreg = LinearRegression()
linreg.fit(X, y)

# Imprimimos el intercepto y los coeficientes
print("Intercepto (β₀):", linreg.intercept_)
print("Coeficientes (β):", linreg.coef_)

# Asociamos cada feature con su coeficiente
list(zip(feature_cols, linreg.coef_))


Se agregaron múltiples variables, en este caso humedad y temperatura para construir una **Regresión lineal múltiple** para ver cómo cada una de estas variables influye individualmente el el número total de alquileres de bicicletas controlando el efecto de la otra.

El signo del coeficiente es directamente proporcional a la variable objetivo, a mayor coeficience mayor alquiler, si es negativo, disminución de alquileres.


Como saber cual modelo funciona mejor?
Para saber cual variable o cuales son las mas utiles y aportan utilidad es necesario usar métricas de evaluación sobre los datos de entrenamiento y prueba.

Estas son:

MAE (Mean absolute error) Es el promedio del error absoluto.
MSE (Mean Squared Error) Es el promedio del error al cuadrado.
RMSE (Root mean squared Error) Raiz cuadrada del MSE
R² Es la varianza del modelo, si es cercana a 1 es mejor el valor


In [ ]:
#Respecto a mi modelo de alquiler de bicicletas
#Importamos librerias necesarias

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

# Definimos las features y la variable objetivo
feature_cols = ['temp', 'humidity']
X = data[feature_cols]
y = data.totally

# Split: 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Entrenamos el modelo
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Predecimos sobre el test set
y_pred = linreg.predict(X_test)

# Evaluamos el rendimiento
print("MAE:", metrics.mean_absolute_error(y_test, y_pred))
print("MSE:", metrics.mean_squared_error(y_test, y_pred))
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("R²:", metrics.r2_score(y_test, y_pred))


MAE = 116.5 En promedio el modelo se equivoca en unos 116 alquileres, sin importar si predijo más o de menos.

MSE = 24,501.51 Al elevar los errores al cuadrado,se penalizan más los errores grandes. Sirve para comparar modelos.

RMSE = 156.53 Error típico del modelo de 157 bicicletas.

R² = 0.25 El modelo sustenta que solo hay un 25% de la variabilidad en el npumero total de los alquileres explicado por las variables que elegimos. 
O sea podemos mejorar el modelo incluyendo mas variables o probando mas algoritmos.

#Comparación de modelos con Train/Test Split y RMSE

Para comparar diferentes modelos de regresión líneal se usa la métrica RMSE en conjunto de entrenamiento y prueba, lo que nos indica que tan bien predice un modelo con datos nuevos, para ver si generaliza bien o solo memoriza datos de entrenamiento


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

def train_test_rmse(feature_cols):
    #Evalúa el desempeño de un modelo de regresión lineal entrenado con ciertas variables predictoras,
    #utilizando train/test split y devolviendo el RMSE sobre el conjunto de prueba.
    
    X = data[feature_cols]
    y = data.totally
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)  # shuffle=False por orden temporal
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

# Comparación con diferentes combinaciones de features
print(train_test_rmse(['temp', 'humidity']))              # → 209.77
print(train_test_rmse(['temp', 'humidity', 'windspeed'])) # → 209.76
print(train_test_rmse(['casual', 'registered']))          # → 2.7e-13



209.7688918306692 Error típico con esas 2 variables
209.76176410054475 El viento no aporta mucho en este modelo
2.718070657801103e-13 Un valor muy bueno para ser real

Este modelo tiene un RMSE casi cero porque usa 'casual' y 'registered', que al sumarse forman exactamente la variable objetivo 'total'.
Esto es un ejemplo de data leakage(Filtración de datos): el modelo conoce la respuesta de antemano.


In [ ]:
#verificamos la suma
# Verificamos que la suma es exacta
np.all(data.casual + data.registered == data.totally)


Para tener en cuenta en el uso de scikit-learn es que sólo acepta features (características) numéricas, por lo que es necesario que aqueelas variables categóricas sean convertidas antes del entrenamiento.

Si la variable categórica es ordenada es posible dejarlo como números que represente el orde y si son no ordenadas acudimos al uso de variables dummies (0 y 1).

En este dataset de bicis la variable del clima es ordenada ya que es codificada númericamente, pero a su vez, tenemos 2 variables no ordenadas, lo son season y holiday y workingday(las cuales ya tienen codificaciones de 0 y 1).

Para crearle variables dummy a season no podemos dejar como primavera=1, verano = 2, ya que no hay un orden claro de las mismas, por lo que en estos casos es necesario usar **pd.get_dummies()**.

Es importante saber que cuando una variable tiene diferentes categorías (K) es necesario usar k-1 columnas dummy para evitar una colinealidad perfecta (trampa de variables ficticias o dummy variable trap)

In [ ]:
season_dummies = pd.get_dummies(data.season, prefix='season')
season_dummies.sample(5, random_state=1)


In [ ]:
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)
data = pd.concat([data, season_dummies], axis=1)
data.sample(5)


In [ ]:
feature_cols = ['temp', 'season_2', 'season_3', 'season_4', 'humidity']
X = data[feature_cols]
y = data.totally

linreg = LinearRegression()
linreg.fit(X, y)

list(zip(feature_cols, linreg.coef_))


Usamos season_1 (primavera) como base (al eliminarla, lo que hace que las otras estaciones se comparen con ella).

season_2 (verano): Se alquilan 3.39 bicicletas menos que en primavera.

season_3 (otoño): Se alquilan 41.73 bicicletas menos que en primavera.

season_4 (invierno): Se alquilan 64.4 bicicletas más que en primavera.




In [ ]:
#Observaremos que tan bien predice el modelo con RMSE (Erro promedio de las preddiciones respecto a los valores reales)
print(train_test_rmse(['temp', 'season_2', 'season_3', 'season_4', 'humidity']))


Esto significa que el modelo predice con un error de +- 209 bicletas respecto al valor real.


### Conclusiones

La regresión múltiple logró explicar un mayor porcentaje de la variabilidad en la demanda de bicicletas. Las variables más influyentes fueron la temperatura y si era un día laboral. Estos modelos pueden ser útiles para planificar estrategias de movilidad urbana en función del clima.

